# InvokeAI Colab
Colab NoteBook Created by [licyk](https://github.com/licyk)

Jupyter Notebook 仓库：[licyk/sd-webui-all-in-one](https://github.com/licyk/sd-webui-all-in-one)

这是适用于 [Colab](https://colab.research.google.com) 部署 InvokeAI 的 Jupyter Notebook，使用时请按顺序执行 Jupyter Notebook 单元。

Colab 链接：<a href="https://colab.research.google.com/github/licyk/sd-webui-all-in-one/blob/main/invokeai_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 功能
1. 环境配置：配置安装的 PyTorch 版本、内网穿透的方式，并安装 [InvokeAI](https://github.com/invoke-ai/InvokeAI)，默认设置下已启用`配置环境完成后立即启动 InvokeAI`选项，则安装完成后将直接启动 InvokeAI，并显示访问地址。
2. 下载模型：下载可选列表中的模型（可选）。
3. 自定义模型下载：使用链接下载模型（可选）。
4. 启动 InvokeAI：启动 InvokeAI，并显示访问地址。


## 使用
1. 在 Colab -> 代码执行程序 > 更改运行时类型 -> 硬件加速器 选择`GPU T4`或者其他 GPU。
2. `环境配置`单元中的选项通常不需要修改，保持默认即可。
3. 运行`环境配置`单元，默认设置下已启用`配置环境完成后立即启动 InvokeAI`选项，则环境配置完成后立即启动 InvokeAI，此时将弹出 Google Drive 授权页面，根据提示进行操作。配置完成后将启动 InvokeAI，InvokeAI 的访问地址可在日志中查看。
4. 如果未启用`配置环境完成后立即启动 InvokeAI`选项，需要运行`启动 InvokeAI`单元，此时将弹出 Google Drive 授权页面，根据提示进行操作。配置完成后将启动 InvokeAI，InvokeAI 的访问地址可在日志中查看。
5. 提示词查询工具：[SD 绘画提示词查询](https://licyk.github.io/t/tag)

## 提示
1. Colab 在关机后将会清除所有数据，所以每次重新启动时必须运行`环境配置`单元才能运行`启动 InvokeAI`单元。
2. [Ngrok](https://ngrok.com) 内网穿透在使用前需要填写 Ngrok Token，可在 [Ngrok](https://ngrok.com) 官网获取。
3. 每次启动 Colab 后必须运行`环境配置`单元，才能运行`启动 InvokeAI`单元启动 InvokeAI。
4. 其他功能有自定义模型下载等功能，根据自己的需求进行使用。
5. 运行`启动 InvokeAI`时将弹出 Google Drive 访问授权提示，根据提示进行授权。授权后，使用 InvokeAI 生成的图片将保存在 Google Drive 的`invokeai_output`文件夹中。
6. Colab 的内存可能不足以让 InvokeAI 进行图片生成，导致 InvokeAI 异常关闭。如果出现该问题，可在启动一次 InvokeAI 后，在 Google Drive 的`invokeai_output`文件夹中找到`invokeai.yaml`文件并编辑，在文件中添加一行`keep_ram_copy_of_weights: false`后保存，再重新启动 InvokeAI。
7. InvokeAI 使用教程可阅读：[InvokeAI 部署与使用 ](https://sdnote.netlify.app/guide/invokeai)

In [ ]:
#@title 👇 环境配置
# InvokeAI Manager 功能初始化部分, 通常不需要修改
# 如果需要查看完整代码实现, 可阅读: https://github.com/licyk/sd-webui-all-in-one/blob/main/sd_scripts_ipynb_core.py
#################################################################################################################
# SD_SCRIPTS_IPYNB_CORE_URL, FORCE_DOWNLOAD_CORE 参数可根据需求修改, 通常保持默认即可
SD_SCRIPTS_IPYNB_CORE_URL = "https://github.com/licyk/sd-webui-all-in-one/raw/main/sd_scripts_ipynb_core.py" # InvokeAI Manager 核心下载地址
FORCE_DOWNLOAD_CORE = False # 设置为 True 时, 即使 InvokeAI Manager 已存在也会重新下载
#################################################################################################################
import os
import sys
import urllib.request
from pathlib import Path
try:
    print(f"[Manager] Jupyter Notebook 根目录: {JUPYTER_ROOT_PATH}") # type: ignore
except Exception as _:
    JUPYTER_ROOT_PATH = Path(os.getcwd())
    sys.path.append(str(JUPYTER_ROOT_PATH.resolve()))
    print(f"[Manager] Jupyter Notebook 根目录: {JUPYTER_ROOT_PATH}")
    SD_SCRIPTS_IPYNB_CORE_PATH = JUPYTER_ROOT_PATH / "sd_scripts_ipynb_core.py" # InvokeAI Manager 核心保存路径
try:
    if SD_SCRIPTS_IPYNB_CORE_PATH.exists() and not FORCE_DOWNLOAD_CORE:
        print("[Manager] InvokeAI Manager 核心模块已存在")
    else:
        urllib.request.urlretrieve(SD_SCRIPTS_IPYNB_CORE_URL, SD_SCRIPTS_IPYNB_CORE_PATH)
        print("[Manager] InvokeAI Manager 核心模块下载成功")
except Exception as e:
    raise Exception(f"InvokeAI Manager 核心模块下载错误: {e}")
from sd_scripts_ipynb_core import logger, VERSION, InvokeAIManager
logger.info("InvokeAI Manager 版本: %s", VERSION)
logger.info("核心模块初始化完成")

#######################################################

#@markdown ## PyTorch 组件版本选项：
#@markdown - PyTorch：
DEVICE_TYPE = "cuda" # @param ["cuda", "rocm", "xpu", "cpu"]
#@markdown ---
#@markdown ## 包管理器选项：
#@markdown - 使用 uv 作为 Python 包管理器
USE_UV = True #@param {type:"boolean"}
#@markdown - PyPI 主镜像源
PIP_INDEX_MIRROR = "https://pypi.python.org/simple" #@param {type:"string"}
#@markdown - PyPI 扩展镜像源
PIP_EXTRA_INDEX_MIRROR = "https://download.pytorch.org/whl/cu124" #@param {type:"string"}
#@markdown - PyTorch 镜像源
PYTORCH_MIRROR = "https://download.pytorch.org/whl/cu124" #@param {type:"string"}
#@markdown ---
#@markdown ## 内网穿透选项：
#@markdown - 使用 remote.moe 内网穿透
USE_REMOTE_MOE = True #@param {type:"boolean"}
#@markdown - 使用 localhost.run 内网穿透
USE_LOCALHOST_RUN = True #@param {type:"boolean"}
#@markdown - 使用 pinggy.io 内网穿透
USE_PINGGY_IO = True #@param {type:"boolean"}
#@markdown - 使用 CloudFlare 内网穿透
USE_CLOUDFLARE = True #@param {type:"boolean"}
#@markdown - 使用 Gradio 内网穿透
USE_GRADIO = True #@param {type:"boolean"}
#@markdown - 使用 Ngrok 内网穿透（需填写 Ngrok Token，可在 [Ngrok](https://ngrok.com) 官网获取）
USE_NGROK = True #@param {type:"boolean"}
#@markdown - Ngrok Token
NGROK_TOKEN = ""  #@param {type:"string"}
#@markdown ---
#@markdown ## 快速启动选项：
#@markdown - 配置环境完成后立即启动 InvokeAI（并挂载 Google Drive）
QUICK_LAUNCH = True #@param {type:"boolean"}
#@markdown - 不重复配置环境（当重复运行环境配置时将不会再进行安装）
NO_REPEAT_CONFIGURE_ENV = True #@param {type:"boolean"}
#@markdown ---
#@markdown ## 其他选项：
#@markdown - 清理无用日志
CLEAR_LOG = True #@param {type:"boolean"}
#@markdown - 检查可用 GPU
CHECK_AVALIABLE_GPU = True #@param {type:"boolean"}
#@markdown - 启用 TCMalloc 内存优化
ENABLE_TCMALLOC = True #@param {type:"boolean"}
#@markdown - 启用 CUDA Malloc 显存优化
ENABLE_CUDA_MALLOC = True #@param {type:"boolean"}
#@markdown - 自动导入模型到 InvokeAI 中
AUTO_IMPORT_MODEL = True #@param {type:"boolean"}

# @markdown ---
##############################################################################
# @markdown ## 模型设置, 在安装时将会下载选择的模型：
SD_MODEL = []

# @markdown - Stable Diffusion 模型
v1_5_pruned_emaonly = False # @param  {type:"boolean"}
animefull_final_pruned = False # @param  {type:"boolean"}
sd_xl_base_1_0_0_9vae = False # @param  {type:"boolean"}
sd_xl_refiner_1_0_0_9vae = False # @param  {type:"boolean"}
sd_xl_turbo_1_0_fp16 = False # @param  {type:"boolean"}
animagine_xl_3_0_base = False # @param  {type:"boolean"}
animagine_xl_3_0 = False # @param  {type:"boolean"}
animagine_xl_3_1 = False # @param  {type:"boolean"}
animagine_xl_4_0 = False # @param  {type:"boolean"}
animagine_xl_4_0_opt = False # @param  {type:"boolean"}
holodayo_xl_2_1 = False # @param  {type:"boolean"}
kivotos_xl_2_0 = False # @param  {type:"boolean"}
clandestine_xl_1_0 = False # @param  {type:"boolean"}
UrangDiffusion_1_1 = False # @param  {type:"boolean"}
RaeDiffusion_XL_v2 = False # @param  {type:"boolean"}
kohaku_xl_delta_rev1 = False # @param  {type:"boolean"}
kohakuXLEpsilon_rev1 = False # @param  {type:"boolean"}
kohaku_xl_epsilon_rev2 = False # @param  {type:"boolean"}
kohaku_xl_epsilon_rev3 = False # @param  {type:"boolean"}
kohaku_xl_zeta = False # @param  {type:"boolean"}
starryXLV52_v52 = False # @param  {type:"boolean"}
heartOfAppleXL_v20 = False # @param  {type:"boolean"}
heartOfAppleXL_v30 = False # @param  {type:"boolean"}
sanaexlAnimeV10_v10 = False # @param  {type:"boolean"}
sanaexlAnimeV10_v11 = False # @param  {type:"boolean"}
SanaeXL_Anime_v1_2_aesthetic = False # @param  {type:"boolean"}
SanaeXL_Anime_v1_3_aesthetic = False # @param  {type:"boolean"}
Illustrious_XL_v0_1 = False # @param  {type:"boolean"}
Illustrious_XL_v0_1_GUIDED = False # @param  {type:"boolean"}
Illustrious_XL_v1_0 = False # @param  {type:"boolean"}
Illustrious_XL_v1_1 = False # @param  {type:"boolean"}
Illustrious_XL_v2_0_stable = False # @param  {type:"boolean"}
jruTheJourneyRemains_v25XL = False # @param  {type:"boolean"}
noobaiXLNAIXL_earlyAccessVersion = False # @param  {type:"boolean"}
noobaiXLNAIXL_epsilonPred05Version = False # @param  {type:"boolean"}
noobaiXLNAIXL_epsilonPred075 = False # @param  {type:"boolean"}
noobaiXLNAIXL_epsilonPred077 = False # @param  {type:"boolean"}
noobaiXLNAIXL_epsilonPred10Version = False # @param  {type:"boolean"}
noobaiXLNAIXL_epsilonPred11Version = False # @param  {type:"boolean"}
noobaiXLNAIXL_vPredTestVersion = False # @param  {type:"boolean"}
noobaiXLNAIXL_vPred05Version = False # @param  {type:"boolean"}
noobaiXLNAIXL_vPred06Version = False # @param  {type:"boolean"}
noobaiXLNAIXL_vPred065SVersion = False # @param  {type:"boolean"}
noobaiXLNAIXL_vPred075SVersion = False # @param  {type:"boolean"}
noobaiXLNAIXL_vPred09RVersion = False # @param  {type:"boolean"}
noobaiXLNAIXL_vPred10Version = True # @param  {type:"boolean"}
ponyDiffusionV6XL_v6StartWithThisOne = False # @param  {type:"boolean"}
pdForAnime_v20 = False # @param  {type:"boolean"}
omegaPonyXLAnime_v20 = False # @param  {type:"boolean"}
# @markdown - VAE 模型
vae_ft_ema_560000_ema_pruned = False # @param  {type:"boolean"}
vae_ft_mse_840000_ema_pruned = False # @param  {type:"boolean"}
sdxl_fp16_fix_vae = True # @param  {type:"boolean"}


v1_5_pruned_emaonly and SD_MODEL.append("https://huggingface.co/licyk/sd-model/resolve/main/sd_1.5/v1-5-pruned-emaonly.safetensors")
animefull_final_pruned and SD_MODEL.append("https://huggingface.co/licyk/sd-model/resolve/main/sd_1.5/animefull-final-pruned.safetensors")
sd_xl_base_1_0_0_9vae and SD_MODEL.append("https://huggingface.co/licyk/sd-model/resolve/main/sdxl_1.0/sd_xl_base_1.0_0.9vae.safetensors")
sd_xl_refiner_1_0_0_9vae and SD_MODEL.append("https://huggingface.co/licyk/sd-model/resolve/main/sdxl_1.0/sd_xl_refiner_1.0_0.9vae.safetensors")
sd_xl_turbo_1_0_fp16 and SD_MODEL.append("https://huggingface.co/licyk/sd-model/resolve/main/sdxl_1.0/sd_xl_turbo_1.0_fp16.safetensors")
animagine_xl_3_0_base and SD_MODEL.append("https://huggingface.co/licyk/sd-model/resolve/main/sdxl_1.0/animagine-xl-3.0-base.safetensors")
animagine_xl_3_0 and SD_MODEL.append("https://huggingface.co/licyk/sd-model/resolve/main/sdxl_1.0/animagine-xl-3.0.safetensors")
animagine_xl_3_1 and SD_MODEL.append("https://huggingface.co/licyk/sd-model/resolve/main/sdxl_1.0/animagine-xl-3.1.safetensors")
animagine_xl_4_0 and SD_MODEL.append("https://huggingface.co/licyk/sd-model/resolve/main/sdxl_1.0/animagine-xl-4.0.safetensors")
animagine_xl_4_0_opt and SD_MODEL.append("https://huggingface.co/licyk/sd-model/resolve/main/sdxl_1.0/animagine-xl-4.0-opt.safetensors")
holodayo_xl_2_1 and SD_MODEL.append("https://huggingface.co/licyk/sd-model/resolve/main/sdxl_1.0/holodayo-xl-2.1.safetensors")
kivotos_xl_2_0 and SD_MODEL.append("https://huggingface.co/licyk/sd-model/resolve/main/sdxl_1.0/kivotos-xl-2.0.safetensors")
clandestine_xl_1_0 and SD_MODEL.append("https://huggingface.co/licyk/sd-model/resolve/main/sdxl_1.0/clandestine-xl-1.0.safetensors")
UrangDiffusion_1_1 and SD_MODEL.append("https://huggingface.co/licyk/sd-model/resolve/main/sdxl_1.0/UrangDiffusion-1.1.safetensors")
RaeDiffusion_XL_v2 and SD_MODEL.append("https://huggingface.co/licyk/sd-model/resolve/main/sdxl_1.0/RaeDiffusion-XL-v2.safetensors")
kohaku_xl_delta_rev1 and SD_MODEL.append("https://huggingface.co/licyk/sd-model/resolve/main/sdxl_1.0/kohaku-xl-delta-rev1.safetensors")
kohakuXLEpsilon_rev1 and SD_MODEL.append("https://huggingface.co/licyk/sd-model/resolve/main/sdxl_1.0/kohakuXLEpsilon_rev1.safetensors")
kohaku_xl_epsilon_rev2 and SD_MODEL.append("https://huggingface.co/licyk/sd-model/resolve/main/sdxl_1.0/kohaku-xl-epsilon-rev2.safetensors")
kohaku_xl_epsilon_rev3 and SD_MODEL.append("https://huggingface.co/licyk/sd-model/resolve/main/sdxl_1.0/kohaku-xl-epsilon-rev3.safetensors")
kohaku_xl_zeta and SD_MODEL.append("https://huggingface.co/licyk/sd-model/resolve/main/sdxl_1.0/kohaku-xl-zeta.safetensors")
starryXLV52_v52 and SD_MODEL.append("https://huggingface.co/licyk/sd-model/resolve/main/sdxl_1.0/starryXLV52_v52.safetensors")
heartOfAppleXL_v20 and SD_MODEL.append("https://huggingface.co/licyk/sd-model/resolve/main/sdxl_1.0/heartOfAppleXL_v20.safetensors")
heartOfAppleXL_v30 and SD_MODEL.append("https://huggingface.co/licyk/sd-model/resolve/main/sdxl_1.0/heartOfAppleXL_v30.safetensors")
sanaexlAnimeV10_v10 and SD_MODEL.append("https://huggingface.co/licyk/sd-model/resolve/main/sdxl_1.0/sanaexlAnimeV10_v10.safetensors")
sanaexlAnimeV10_v11 and SD_MODEL.append("https://huggingface.co/licyk/sd-model/resolve/main/sdxl_1.0/sanaexlAnimeV10_v11.safetensors")
SanaeXL_Anime_v1_2_aesthetic and SD_MODEL.append("https://huggingface.co/licyk/sd-model/resolve/main/sdxl_1.0/SanaeXL-Anime-v1.2-aesthetic.safetensors")
SanaeXL_Anime_v1_3_aesthetic and SD_MODEL.append("https://huggingface.co/licyk/sd-model/resolve/main/sdxl_1.0/SanaeXL-Anime-v1.3-aesthetic.safetensors")
Illustrious_XL_v0_1 and SD_MODEL.append("https://huggingface.co/licyk/sd-model/resolve/main/sdxl_1.0/Illustrious-XL-v0.1.safetensors")
Illustrious_XL_v0_1_GUIDED and SD_MODEL.append("https://huggingface.co/licyk/sd-model/resolve/main/sdxl_1.0/Illustrious-XL-v0.1-GUIDED.safetensors")
Illustrious_XL_v1_0 and SD_MODEL.append("https://huggingface.co/licyk/sd-model/resolve/main/sdxl_1.0/Illustrious-XL-v1.0.safetensors")
Illustrious_XL_v1_1 and SD_MODEL.append("https://huggingface.co/licyk/sd-model/resolve/main/sdxl_1.0/Illustrious-XL-v1.1.safetensors")
Illustrious_XL_v2_0_stable and SD_MODEL.append("https://huggingface.co/licyk/sd-model/resolve/main/sdxl_1.0/Illustrious-XL-v2.0-stable.safetensors")
jruTheJourneyRemains_v25XL and SD_MODEL.append("https://huggingface.co/licyk/sd-model/resolve/main/sdxl_1.0/jruTheJourneyRemains_v25XL.safetensors")
noobaiXLNAIXL_earlyAccessVersion and SD_MODEL.append("https://huggingface.co/licyk/sd-model/resolve/main/sdxl_1.0/noobaiXLNAIXL_earlyAccessVersion.safetensors")
noobaiXLNAIXL_epsilonPred05Version and SD_MODEL.append("https://huggingface.co/licyk/sd-model/resolve/main/sdxl_1.0/noobaiXLNAIXL_epsilonPred05Version.safetensors")
noobaiXLNAIXL_epsilonPred075 and SD_MODEL.append("https://huggingface.co/licyk/sd-model/resolve/main/sdxl_1.0/noobaiXLNAIXL_epsilonPred075.safetensors")
noobaiXLNAIXL_epsilonPred077 and SD_MODEL.append("https://huggingface.co/licyk/sd-model/resolve/main/sdxl_1.0/noobaiXLNAIXL_epsilonPred077.safetensors")
noobaiXLNAIXL_epsilonPred10Version and SD_MODEL.append("https://huggingface.co/licyk/sd-model/resolve/main/sdxl_1.0/noobaiXLNAIXL_epsilonPred10Version.safetensors")
noobaiXLNAIXL_epsilonPred11Version and SD_MODEL.append("https://huggingface.co/licyk/sd-model/resolve/main/sdxl_1.0/noobaiXLNAIXL_epsilonPred11Version.safetensors")
noobaiXLNAIXL_vPredTestVersion and SD_MODEL.append("https://huggingface.co/licyk/sd-model/resolve/main/sdxl_1.0/noobaiXLNAIXL_vPredTestVersion.safetensors")
noobaiXLNAIXL_vPred05Version and SD_MODEL.append("https://huggingface.co/licyk/sd-model/resolve/main/sdxl_1.0/noobaiXLNAIXL_vPred05Version.safetensors")
noobaiXLNAIXL_vPred06Version and SD_MODEL.append("https://huggingface.co/licyk/sd-model/resolve/main/sdxl_1.0/noobaiXLNAIXL_vPred06Version.safetensors")
noobaiXLNAIXL_vPred065SVersion and SD_MODEL.append("https://huggingface.co/licyk/sd-model/resolve/main/sdxl_1.0/noobaiXLNAIXL_vPred065SVersion.safetensors")
noobaiXLNAIXL_vPred075SVersion and SD_MODEL.append("https://huggingface.co/licyk/sd-model/resolve/main/sdxl_1.0/noobaiXLNAIXL_vPred075SVersion.safetensors")
noobaiXLNAIXL_vPred09RVersion and SD_MODEL.append("https://huggingface.co/licyk/sd-model/resolve/main/sdxl_1.0/noobaiXLNAIXL_vPred09RVersion.safetensors")
noobaiXLNAIXL_vPred10Version and SD_MODEL.append("https://huggingface.co/licyk/sd-model/resolve/main/sdxl_1.0/noobaiXLNAIXL_vPred10Version.safetensors")
ponyDiffusionV6XL_v6StartWithThisOne and SD_MODEL.append("https://huggingface.co/licyk/sd-model/resolve/main/sdxl_1.0/ponyDiffusionV6XL_v6StartWithThisOne.safetensors")
pdForAnime_v20 and SD_MODEL.append("https://huggingface.co/licyk/sd-model/resolve/main/sdxl_1.0/pdForAnime_v20.safetensors")
omegaPonyXLAnime_v20 and SD_MODEL.append("https://huggingface.co/licyk/sd-model/resolve/main/sdxl_1.0/omegaPonyXLAnime_v20.safetensors")
vae_ft_ema_560000_ema_pruned and SD_MODEL.append("https://huggingface.co/licyk/sd-vae/resolve/main/sd_1.5/vae-ft-ema-560000-ema-pruned.safetensors")
vae_ft_mse_840000_ema_pruned and SD_MODEL.append("https://huggingface.co/licyk/sd-vae/resolve/main/sd_1.5/vae-ft-mse-840000-ema-pruned.safetensors")
sdxl_fp16_fix_vae and SD_MODEL.append("https://huggingface.co/licyk/sd-vae/resolve/main/sdxl_1.0/sdxl_fp16_fix_vae.safetensors")

##############################################################################

INSTALL_PARAMS = {
    "device_type": DEVICE_TYPE,
    "use_uv": USE_UV,
    "pypi_index_mirror": PIP_INDEX_MIRROR or None,
    "pypi_extra_index_mirror": PIP_EXTRA_INDEX_MIRROR or None,
    # Colab 的环境暂不需要以下镜像源
    # "pypi_find_links_mirror": PIP_FIND_LINKS_MIRROR or None,
    # "github_mirror": GITHUB_MIRROR or None,
    # "huggingface_mirror": HUGGINGFACE_MIRROR or None,
    # 使用 InvokeAI Manager 内置 PyTorch 镜像源字典
    # "pytorch_mirror_dict": PYTORCH_MIRROR_DICT or None,
    "model_list": SD_MODEL,
    "check_avaliable_gpu": CHECK_AVALIABLE_GPU,
    "enable_tcmalloc": ENABLE_TCMALLOC,
    "enable_cuda_malloc": ENABLE_CUDA_MALLOC,
}

TUNNEL_PARAMS = {
    "use_ngrok": USE_NGROK,
    "ngrok_token": NGROK_TOKEN or None,
    "use_cloudflare": USE_CLOUDFLARE,
    "use_remote_moe": USE_REMOTE_MOE,
    "use_localhost_run": USE_LOCALHOST_RUN,
    "use_gradio": USE_GRADIO,
    "use_pinggy_io": USE_PINGGY_IO,
    "message": "##### InvokeAI 访问地址 #####",
}

INVOKEAI_PATH = "/content/InvokeAI"
invokeai = InvokeAIManager(os.path.dirname(INVOKEAI_PATH), os.path.basename(INVOKEAI_PATH), port=9090)

try:
    _ = invokeai_has_init # type: ignore
except:
    invokeai_has_init = False

if NO_REPEAT_CONFIGURE_ENV:
    if not invokeai_has_init:
        invokeai.install(**INSTALL_PARAMS)
        INIT_CONFIG = 1
        invokeai_has_init = True
        CLEAR_LOG and invokeai.utils.clear_up()
        logger.info("InvokeAI 运行环境配置完成")
    else:
        logger.info("检测到不重复配置环境已启用, 并且在当前 Colab 实例中已配置 InvokeAI 运行环境, 不再重复配置 InvokeAI 运行环境")
        logger.info("如需在当前 Colab 实例中重新配置 InvokeAI 运行环境, 请在快速启动选项中取消不重复配置环境功能")
else:
    invokeai.install(**INSTALL_PARAMS)
    INIT_CONFIG = 1
    CLEAR_LOG and invokeai.utils.clear_up()
    logger.info("InvokeAI 运行环境配置完成")

if QUICK_LAUNCH:
    logger.info("启动 InvokeAI 中")
    os.chdir(INVOKEAI_PATH)
    invokeai.check_env(use_uv=USE_UV)
    invokeai.mount_drive()
    if AUTO_IMPORT_MODEL:
        invokeai.import_model()
    invokeai.tun.start_tunnel(**TUNNEL_PARAMS)
    !python -c "from invokeai.app.run_app import run_app; run_app()"
    os.chdir(JUPYTER_ROOT_PATH)
    CLEAR_LOG and invokeai.utils.clear_up()
    logger.info("已关闭 InvokeAI")

In [ ]:
#@title 👇 下载模型（可选）

try:
    _ = INIT_CONFIG
except:
    raise Exception("未安装 InvokeAI")

#@markdown 选择下载的模型：
##############################
SD_MODEL = []

# @markdown - Stable Diffusion 模型
v1_5_pruned_emaonly = False # @param  {type:"boolean"}
animefull_final_pruned = False # @param  {type:"boolean"}
sd_xl_base_1_0_0_9vae = False # @param  {type:"boolean"}
sd_xl_refiner_1_0_0_9vae = False # @param  {type:"boolean"}
sd_xl_turbo_1_0_fp16 = False # @param  {type:"boolean"}
animagine_xl_3_0_base = False # @param  {type:"boolean"}
animagine_xl_3_0 = False # @param  {type:"boolean"}
animagine_xl_3_1 = False # @param  {type:"boolean"}
animagine_xl_4_0 = False # @param  {type:"boolean"}
animagine_xl_4_0_opt = False # @param  {type:"boolean"}
holodayo_xl_2_1 = False # @param  {type:"boolean"}
kivotos_xl_2_0 = False # @param  {type:"boolean"}
clandestine_xl_1_0 = False # @param  {type:"boolean"}
UrangDiffusion_1_1 = False # @param  {type:"boolean"}
RaeDiffusion_XL_v2 = False # @param  {type:"boolean"}
kohaku_xl_delta_rev1 = False # @param  {type:"boolean"}
kohakuXLEpsilon_rev1 = False # @param  {type:"boolean"}
kohaku_xl_epsilon_rev2 = False # @param  {type:"boolean"}
kohaku_xl_epsilon_rev3 = False # @param  {type:"boolean"}
kohaku_xl_zeta = False # @param  {type:"boolean"}
starryXLV52_v52 = False # @param  {type:"boolean"}
heartOfAppleXL_v20 = False # @param  {type:"boolean"}
heartOfAppleXL_v30 = False # @param  {type:"boolean"}
sanaexlAnimeV10_v10 = False # @param  {type:"boolean"}
sanaexlAnimeV10_v11 = False # @param  {type:"boolean"}
SanaeXL_Anime_v1_2_aesthetic = False # @param  {type:"boolean"}
SanaeXL_Anime_v1_3_aesthetic = False # @param  {type:"boolean"}
Illustrious_XL_v0_1 = False # @param  {type:"boolean"}
Illustrious_XL_v0_1_GUIDED = False # @param  {type:"boolean"}
Illustrious_XL_v1_0 = False # @param  {type:"boolean"}
Illustrious_XL_v1_1 = False # @param  {type:"boolean"}
Illustrious_XL_v2_0_stable = False # @param  {type:"boolean"}
jruTheJourneyRemains_v25XL = False # @param  {type:"boolean"}
noobaiXLNAIXL_earlyAccessVersion = False # @param  {type:"boolean"}
noobaiXLNAIXL_epsilonPred05Version = False # @param  {type:"boolean"}
noobaiXLNAIXL_epsilonPred075 = False # @param  {type:"boolean"}
noobaiXLNAIXL_epsilonPred077 = False # @param  {type:"boolean"}
noobaiXLNAIXL_epsilonPred10Version = False # @param  {type:"boolean"}
noobaiXLNAIXL_epsilonPred11Version = False # @param  {type:"boolean"}
noobaiXLNAIXL_vPredTestVersion = False # @param  {type:"boolean"}
noobaiXLNAIXL_vPred05Version = False # @param  {type:"boolean"}
noobaiXLNAIXL_vPred06Version = False # @param  {type:"boolean"}
noobaiXLNAIXL_vPred065SVersion = False # @param  {type:"boolean"}
noobaiXLNAIXL_vPred075SVersion = False # @param  {type:"boolean"}
noobaiXLNAIXL_vPred09RVersion = False # @param  {type:"boolean"}
noobaiXLNAIXL_vPred10Version = False # @param  {type:"boolean"}
ponyDiffusionV6XL_v6StartWithThisOne = False # @param  {type:"boolean"}
pdForAnime_v20 = False # @param  {type:"boolean"}
omegaPonyXLAnime_v20 = False # @param  {type:"boolean"}
# @markdown - VAE 模型
vae_ft_ema_560000_ema_pruned = False # @param  {type:"boolean"}
vae_ft_mse_840000_ema_pruned = False # @param  {type:"boolean"}
sdxl_fp16_fix_vae = False # @param  {type:"boolean"}

v1_5_pruned_emaonly and SD_MODEL.append("https://huggingface.co/licyk/sd-model/resolve/main/sd_1.5/v1-5-pruned-emaonly.safetensors")
animefull_final_pruned and SD_MODEL.append("https://huggingface.co/licyk/sd-model/resolve/main/sd_1.5/animefull-final-pruned.safetensors")
sd_xl_base_1_0_0_9vae and SD_MODEL.append("https://huggingface.co/licyk/sd-model/resolve/main/sdxl_1.0/sd_xl_base_1.0_0.9vae.safetensors")
sd_xl_refiner_1_0_0_9vae and SD_MODEL.append("https://huggingface.co/licyk/sd-model/resolve/main/sdxl_1.0/sd_xl_refiner_1.0_0.9vae.safetensors")
sd_xl_turbo_1_0_fp16 and SD_MODEL.append("https://huggingface.co/licyk/sd-model/resolve/main/sdxl_1.0/sd_xl_turbo_1.0_fp16.safetensors")
animagine_xl_3_0_base and SD_MODEL.append("https://huggingface.co/licyk/sd-model/resolve/main/sdxl_1.0/animagine-xl-3.0-base.safetensors")
animagine_xl_3_0 and SD_MODEL.append("https://huggingface.co/licyk/sd-model/resolve/main/sdxl_1.0/animagine-xl-3.0.safetensors")
animagine_xl_3_1 and SD_MODEL.append("https://huggingface.co/licyk/sd-model/resolve/main/sdxl_1.0/animagine-xl-3.1.safetensors")
animagine_xl_4_0 and SD_MODEL.append("https://huggingface.co/licyk/sd-model/resolve/main/sdxl_1.0/animagine-xl-4.0.safetensors")
animagine_xl_4_0_opt and SD_MODEL.append("https://huggingface.co/licyk/sd-model/resolve/main/sdxl_1.0/animagine-xl-4.0-opt.safetensors")
holodayo_xl_2_1 and SD_MODEL.append("https://huggingface.co/licyk/sd-model/resolve/main/sdxl_1.0/holodayo-xl-2.1.safetensors")
kivotos_xl_2_0 and SD_MODEL.append("https://huggingface.co/licyk/sd-model/resolve/main/sdxl_1.0/kivotos-xl-2.0.safetensors")
clandestine_xl_1_0 and SD_MODEL.append("https://huggingface.co/licyk/sd-model/resolve/main/sdxl_1.0/clandestine-xl-1.0.safetensors")
UrangDiffusion_1_1 and SD_MODEL.append("https://huggingface.co/licyk/sd-model/resolve/main/sdxl_1.0/UrangDiffusion-1.1.safetensors")
RaeDiffusion_XL_v2 and SD_MODEL.append("https://huggingface.co/licyk/sd-model/resolve/main/sdxl_1.0/RaeDiffusion-XL-v2.safetensors")
kohaku_xl_delta_rev1 and SD_MODEL.append("https://huggingface.co/licyk/sd-model/resolve/main/sdxl_1.0/kohaku-xl-delta-rev1.safetensors")
kohakuXLEpsilon_rev1 and SD_MODEL.append("https://huggingface.co/licyk/sd-model/resolve/main/sdxl_1.0/kohakuXLEpsilon_rev1.safetensors")
kohaku_xl_epsilon_rev2 and SD_MODEL.append("https://huggingface.co/licyk/sd-model/resolve/main/sdxl_1.0/kohaku-xl-epsilon-rev2.safetensors")
kohaku_xl_epsilon_rev3 and SD_MODEL.append("https://huggingface.co/licyk/sd-model/resolve/main/sdxl_1.0/kohaku-xl-epsilon-rev3.safetensors")
kohaku_xl_zeta and SD_MODEL.append("https://huggingface.co/licyk/sd-model/resolve/main/sdxl_1.0/kohaku-xl-zeta.safetensors")
starryXLV52_v52 and SD_MODEL.append("https://huggingface.co/licyk/sd-model/resolve/main/sdxl_1.0/starryXLV52_v52.safetensors")
heartOfAppleXL_v20 and SD_MODEL.append("https://huggingface.co/licyk/sd-model/resolve/main/sdxl_1.0/heartOfAppleXL_v20.safetensors")
heartOfAppleXL_v30 and SD_MODEL.append("https://huggingface.co/licyk/sd-model/resolve/main/sdxl_1.0/heartOfAppleXL_v30.safetensors")
sanaexlAnimeV10_v10 and SD_MODEL.append("https://huggingface.co/licyk/sd-model/resolve/main/sdxl_1.0/sanaexlAnimeV10_v10.safetensors")
sanaexlAnimeV10_v11 and SD_MODEL.append("https://huggingface.co/licyk/sd-model/resolve/main/sdxl_1.0/sanaexlAnimeV10_v11.safetensors")
SanaeXL_Anime_v1_2_aesthetic and SD_MODEL.append("https://huggingface.co/licyk/sd-model/resolve/main/sdxl_1.0/SanaeXL-Anime-v1.2-aesthetic.safetensors")
SanaeXL_Anime_v1_3_aesthetic and SD_MODEL.append("https://huggingface.co/licyk/sd-model/resolve/main/sdxl_1.0/SanaeXL-Anime-v1.3-aesthetic.safetensors")
Illustrious_XL_v0_1 and SD_MODEL.append("https://huggingface.co/licyk/sd-model/resolve/main/sdxl_1.0/Illustrious-XL-v0.1.safetensors")
Illustrious_XL_v0_1_GUIDED and SD_MODEL.append("https://huggingface.co/licyk/sd-model/resolve/main/sdxl_1.0/Illustrious-XL-v0.1-GUIDED.safetensors")
Illustrious_XL_v1_0 and SD_MODEL.append("https://huggingface.co/licyk/sd-model/resolve/main/sdxl_1.0/Illustrious-XL-v1.0.safetensors")
Illustrious_XL_v1_1 and SD_MODEL.append("https://huggingface.co/licyk/sd-model/resolve/main/sdxl_1.0/Illustrious-XL-v1.1.safetensors")
Illustrious_XL_v2_0_stable and SD_MODEL.append("https://huggingface.co/licyk/sd-model/resolve/main/sdxl_1.0/Illustrious-XL-v2.0-stable.safetensors")
jruTheJourneyRemains_v25XL and SD_MODEL.append("https://huggingface.co/licyk/sd-model/resolve/main/sdxl_1.0/jruTheJourneyRemains_v25XL.safetensors")
noobaiXLNAIXL_earlyAccessVersion and SD_MODEL.append("https://huggingface.co/licyk/sd-model/resolve/main/sdxl_1.0/noobaiXLNAIXL_earlyAccessVersion.safetensors")
noobaiXLNAIXL_epsilonPred05Version and SD_MODEL.append("https://huggingface.co/licyk/sd-model/resolve/main/sdxl_1.0/noobaiXLNAIXL_epsilonPred05Version.safetensors")
noobaiXLNAIXL_epsilonPred075 and SD_MODEL.append("https://huggingface.co/licyk/sd-model/resolve/main/sdxl_1.0/noobaiXLNAIXL_epsilonPred075.safetensors")
noobaiXLNAIXL_epsilonPred077 and SD_MODEL.append("https://huggingface.co/licyk/sd-model/resolve/main/sdxl_1.0/noobaiXLNAIXL_epsilonPred077.safetensors")
noobaiXLNAIXL_epsilonPred10Version and SD_MODEL.append("https://huggingface.co/licyk/sd-model/resolve/main/sdxl_1.0/noobaiXLNAIXL_epsilonPred10Version.safetensors")
noobaiXLNAIXL_epsilonPred11Version and SD_MODEL.append("https://huggingface.co/licyk/sd-model/resolve/main/sdxl_1.0/noobaiXLNAIXL_epsilonPred11Version.safetensors")
noobaiXLNAIXL_vPredTestVersion and SD_MODEL.append("https://huggingface.co/licyk/sd-model/resolve/main/sdxl_1.0/noobaiXLNAIXL_vPredTestVersion.safetensors")
noobaiXLNAIXL_vPred05Version and SD_MODEL.append("https://huggingface.co/licyk/sd-model/resolve/main/sdxl_1.0/noobaiXLNAIXL_vPred05Version.safetensors")
noobaiXLNAIXL_vPred06Version and SD_MODEL.append("https://huggingface.co/licyk/sd-model/resolve/main/sdxl_1.0/noobaiXLNAIXL_vPred06Version.safetensors")
noobaiXLNAIXL_vPred065SVersion and SD_MODEL.append("https://huggingface.co/licyk/sd-model/resolve/main/sdxl_1.0/noobaiXLNAIXL_vPred065SVersion.safetensors")
noobaiXLNAIXL_vPred075SVersion and SD_MODEL.append("https://huggingface.co/licyk/sd-model/resolve/main/sdxl_1.0/noobaiXLNAIXL_vPred075SVersion.safetensors")
noobaiXLNAIXL_vPred09RVersion and SD_MODEL.append("https://huggingface.co/licyk/sd-model/resolve/main/sdxl_1.0/noobaiXLNAIXL_vPred09RVersion.safetensors")
noobaiXLNAIXL_vPred10Version and SD_MODEL.append("https://huggingface.co/licyk/sd-model/resolve/main/sdxl_1.0/noobaiXLNAIXL_vPred10Version.safetensors")
ponyDiffusionV6XL_v6StartWithThisOne and SD_MODEL.append("https://huggingface.co/licyk/sd-model/resolve/main/sdxl_1.0/ponyDiffusionV6XL_v6StartWithThisOne.safetensors")
pdForAnime_v20 and SD_MODEL.append("https://huggingface.co/licyk/sd-model/resolve/main/sdxl_1.0/pdForAnime_v20.safetensors")
omegaPonyXLAnime_v20 and SD_MODEL.append("https://huggingface.co/licyk/sd-model/resolve/main/sdxl_1.0/omegaPonyXLAnime_v20.safetensors")
vae_ft_ema_560000_ema_pruned and SD_MODEL.append("https://huggingface.co/licyk/sd-vae/resolve/main/sd_1.5/vae-ft-ema-560000-ema-pruned.safetensors")
vae_ft_mse_840000_ema_pruned and SD_MODEL.append("https://huggingface.co/licyk/sd-vae/resolve/main/sd_1.5/vae-ft-mse-840000-ema-pruned.safetensors")
sdxl_fp16_fix_vae and SD_MODEL.append("https://huggingface.co/licyk/sd-vae/resolve/main/sdxl_1.0/sdxl_fp16_fix_vae.safetensors")

##############################
logger.info("下载模型中")
invokeai.get_sd_model_from_list(SD_MODEL)
CLEAR_LOG and invokeai.utils.clear_up()
logger.info("模型下载完成")

In [ ]:
#@title 👇 自定义模型下载
try:
    _ = INIT_CONFIG
except:
    raise Exception("未安装 InvokeAI")

#@markdown ### 填写模型的下载链接：
model_url = "https://huggingface.co/licyk/sd-lora/resolve/main/sdxl/style/CoolFlatColor.safetensors"  #@param {type:"string"}
#@markdown ### 填写模型的名称（包括后缀名）：
model_name = "CoolFlatColor.safetensors"  #@param {type:"string"}

invokeai.get_sd_model(
    url=model_url,
    filename=model_name or None
)

In [ ]:
#@title 👇 启动 InvokeAI
try:
    _ = INIT_CONFIG
except:
    raise Exception("未安装 InvokeAI")

logger.info("启动 InvokeAI")
os.chdir(INVOKEAI_PATH)
invokeai.check_env(use_uv=USE_UV)
invokeai.mount_drive()
if AUTO_IMPORT_MODEL:
    invokeai.import_model()
invokeai.tun.start_tunnel(**TUNNEL_PARAMS)
!python -c "from invokeai.app.run_app import run_app; run_app()"
os.chdir(JUPYTER_ROOT_PATH)
CLEAR_LOG and invokeai.utils.clear_up()
logger.info("已关闭 InvokeAI")

In [ ]:
#@title 👇 文件下载工具

#@markdown ### 填写文件的下载链接：
url = ""  #@param {type:"string"}
#@markdown ### 填写文件的保存路径：
file_path = "/content"  #@param {type:"string"}
#@markdown ### 填写文件的保存名称 (可选)：
filename = ""  #@param {type:"string"}

invokeai.downloader.download_file(
    url=url,
    path=file_path or None,
    save_name=filename or None,
)
